In [114]:
from keras.layers import Input, Embedding, SimpleRNN, Dense, merge, Flatten, BatchNormalization, LSTM, TimeDistributed, Dropout
from keras.models import Model
from keras.optimizers import Adam
import urllib2
import numpy as np

dataset_raw = urllib2.urlopen("https://s3.amazonaws.com/text-datasets/nietzsche.txt").read().\
    replace('\n', ' ')

In [115]:
vocab = sorted(list(set([i for i in dataset_raw])))

In [116]:
vocab.insert(0, '\0')

In [117]:
txt_encoder = {v:k for k,v in enumerate(vocab)}
txt_decoder = {k:v for k,v in enumerate(vocab)}

In [118]:
dataset_encoded = [txt_encoder[i] for i in dataset_raw]

In [119]:
vocab_size = len(vocab)

In [120]:
seq_len = 40

In [121]:
train_data = [np.stack([dataset_encoded[i + j] for i in range(0, len(dataset_raw) - seq_len - 1, seq_len)]) for j in range(seq_len)]

In [122]:
output_data = [np.stack([dataset_encoded[i + j]
                         for i in range(0, len(dataset_raw) - seq_len - 1, seq_len)])[:,np.newaxis] 
                         for j in range(1, seq_len + 1)]

In [10]:
inps = []
embs = []

for i in range(seq_len):
    inps.append(Input(shape=(1,), name='inp_%s' % i))
    embs.append(Flatten()(Embedding(input_dim=vocab_size, output_dim=40, name='emb_%s' % i)(inps[i])) )

In [11]:
hidden_layer_size = 256

In [12]:
dense_in = Dense(hidden_layer_size, activation='relu')
dense_hidden = Dense(hidden_layer_size, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [13]:
outs = []

zero_inp = Input(shape=(40,), name='zeros')
hidden = dense_in(zero_inp)

for i in range(seq_len):
    bn = BatchNormalization()(embs[i])
    din = dense_in(bn)
    hidden = merge([din, dense_hidden(hidden)])
    outs.append(dense_out(hidden))

In [14]:
zeros = np.tile(np.zeros(40), (len(train_data[0]), 1))

In [ ]:
inp = Input(shape=(len(train_data[0]),40))
emb = Embedding(vocab_size, 60, input_length=40)

In [15]:
mdl = Model(input=[zero_inp] + [i for i in inps], output=outs)

In [16]:
mdl.compile(optimizer=Adam(lr=0.00001), loss='sparse_categorical_crossentropy')

In [21]:
mdl.fit([zeros] + train_data, output_data, nb_epoch=12, batch_size=64)

Epoch 1/12
75112/75112 [==============================] - 15s - loss: 28.6928 - dense_3_loss_1: 4.2481 - dense_3_loss_2: 4.0295 - dense_3_loss_3: 3.7415 - dense_3_loss_4: 3.4701 - dense_3_loss_5: 3.3294 - dense_3_loss_6: 3.2722 - dense_3_loss_7: 3.2750 - dense_3_loss_8: 3.3271    
Epoch 2/12
75112/75112 [==============================] - 15s - loss: 25.7862 - dense_3_loss_1: 3.9131 - dense_3_loss_2: 3.5009 - dense_3_loss_3: 3.1995 - dense_3_loss_4: 3.0287 - dense_3_loss_5: 3.0105 - dense_3_loss_6: 3.0168 - dense_3_loss_7: 3.0476 - dense_3_loss_8: 3.0690    
Epoch 3/12
75112/75112 [==============================] - 15s - loss: 24.4473 - dense_3_loss_1: 3.6158 - dense_3_loss_2: 3.1935 - dense_3_loss_3: 3.0057 - dense_3_loss_4: 2.9193 - dense_3_loss_5: 2.9235 - dense_3_loss_6: 2.9200 - dense_3_loss_7: 2.9370 - dense_3_loss_8: 2.9325    
Epoch 4/12
75112/75112 [==============================] - 15s - loss: 23.3779 - dense_3_loss_1: 3.3494 - dense_3_loss_2: 2.9923 - dense_3_loss_3: 2.8867 -

In [22]:
mdl.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy')
mdl.fit([zeros] + train_data, output_data, nb_epoch=12, batch_size=64)

Epoch 1/12
75112/75112 [==============================] - 15s - loss: 19.3652 - dense_3_loss_1: 2.5690 - dense_3_loss_2: 2.4519 - dense_3_loss_3: 2.4105 - dense_3_loss_4: 2.3885 - dense_3_loss_5: 2.3888 - dense_3_loss_6: 2.3814 - dense_3_loss_7: 2.3944 - dense_3_loss_8: 2.3807    
Epoch 2/12
75112/75112 [==============================] - 15s - loss: 18.6464 - dense_3_loss_1: 2.5130 - dense_3_loss_2: 2.3812 - dense_3_loss_3: 2.3194 - dense_3_loss_4: 2.2896 - dense_3_loss_5: 2.2908 - dense_3_loss_6: 2.2800 - dense_3_loss_7: 2.2927 - dense_3_loss_8: 2.2797    
Epoch 3/12
75112/75112 [==============================] - 15s - loss: 18.1987 - dense_3_loss_1: 2.4889 - dense_3_loss_2: 2.3438 - dense_3_loss_3: 2.2663 - dense_3_loss_4: 2.2265 - dense_3_loss_5: 2.2253 - dense_3_loss_6: 2.2118 - dense_3_loss_7: 2.2259 - dense_3_loss_8: 2.2102    
Epoch 4/12
75112/75112 [==============================] - 15s - loss: 17.8806 - dense_3_loss_1: 2.4754 - dense_3_loss_2: 2.3204 - dense_3_loss_3: 2.2300 -

In [45]:
mdl.compile(optimizer=Adam(lr=0.01), loss='sparse_categorical_crossentropy')
mdl.fit([zeros] + train_data, output_data, nb_epoch=12, batch_size=64)

Epoch 1/12
75112/75112 [==============================] - 15s - loss: 18.7042 - dense_3_loss_1: 2.5083 - dense_3_loss_2: 2.3797 - dense_3_loss_3: 2.3180 - dense_3_loss_4: 2.2843 - dense_3_loss_5: 2.3040 - dense_3_loss_6: 2.2955 - dense_3_loss_7: 2.3150 - dense_3_loss_8: 2.2993    
Epoch 2/12
75112/75112 [==============================] - 14s - loss: 17.4989 - dense_3_loss_1: 2.4632 - dense_3_loss_2: 2.3154 - dense_3_loss_3: 2.1877 - dense_3_loss_4: 2.1194 - dense_3_loss_5: 2.1143 - dense_3_loss_6: 2.0968 - dense_3_loss_7: 2.1094 - dense_3_loss_8: 2.0927    
Epoch 3/12
75112/75112 [==============================] - 14s - loss: 17.2581 - dense_3_loss_1: 2.4592 - dense_3_loss_2: 2.3099 - dense_3_loss_3: 2.1657 - dense_3_loss_4: 2.0840 - dense_3_loss_5: 2.0709 - dense_3_loss_6: 2.0515 - dense_3_loss_7: 2.0681 - dense_3_loss_8: 2.0488    
Epoch 4/12
75112/75112 [==============================] - 14s - loss: 17.0941 - dense_3_loss_1: 2.4557 - dense_3_loss_2: 2.3046 - dense_3_loss_3: 2.1515 -

In [56]:
mdl.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy')
mdl.fit([zeros] + train_data, output_data, nb_epoch=12, batch_size=64)

Epoch 1/12
75112/75112 [==============================] - 15s - loss: 16.3331 - dense_3_loss_1: 2.4323 - dense_3_loss_2: 2.2662 - dense_3_loss_3: 2.0841 - dense_3_loss_4: 1.9621 - dense_3_loss_5: 1.9194 - dense_3_loss_6: 1.8863 - dense_3_loss_7: 1.9009 - dense_3_loss_8: 1.8819    
Epoch 2/12
75112/75112 [==============================] - 15s - loss: 16.2417 - dense_3_loss_1: 2.4299 - dense_3_loss_2: 2.2624 - dense_3_loss_3: 2.0759 - dense_3_loss_4: 1.9511 - dense_3_loss_5: 1.9065 - dense_3_loss_6: 1.8687 - dense_3_loss_7: 1.8830 - dense_3_loss_8: 1.8642    
Epoch 3/12
75112/75112 [==============================] - 15s - loss: 16.2094 - dense_3_loss_1: 2.4286 - dense_3_loss_2: 2.2617 - dense_3_loss_3: 2.0729 - dense_3_loss_4: 1.9471 - dense_3_loss_5: 1.9016 - dense_3_loss_6: 1.8637 - dense_3_loss_7: 1.8760 - dense_3_loss_8: 1.8580    
Epoch 4/12
75112/75112 [==============================] - 15s - loss: 16.1919 - dense_3_loss_1: 2.4282 - dense_3_loss_2: 2.2610 - dense_3_loss_3: 2.0722 -

In [17]:
def mdl_predict(seq_3char):
    if len(seq_3char) < 8:
        padding_len = 8 - len(seq_3char)
        padding = '\0' * padding_len
        seq_3char = padding + seq_3char
    pred_data = [txt_encoder[i] for i in seq_3char]
    arrs = [np.stack([i]) for i in pred_data]
    pred = mdl.predict([np.tile(np.zeros(40), (1, 1))] + arrs)
    return [txt_decoder[np.argmax(o)] for o in pred]

In [18]:
mdl_predict('sufferin')

[')', ';', ';', ';', ';', ';', ';', ';']

In [111]:
def generate_text(num_chars):
    outs = []
    base_str = 'Sufferin'
    for i in range(num_chars):
        prediction = mdl.predict([np.stack(np.zeros(40))[np.newaxis]] +
                                          [np.array([txt_encoder[i]]) for i in base_str])
        next_char = np.argmax(prediction[-1]) # the final model output
        outs.append(txt_decoder[next_char])
        base_str = (base_str + txt_decoder[next_char])[-40:]
    return base_str

In [75]:
generate_text(100)

';S;;S;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'

In [135]:
inp = Input(batch_shape=(64, seq_len))
emb = Embedding(input_dim=vocab_size, output_dim=40, batch_input_shape=(64,seq_len))(inp)
bn = BatchNormalization()(emb)
rnn = LSTM(output_dim=256, activation='relu', return_sequences=True, stateful=True)(bn)
bn2 = BatchNormalization()(rnn)
d = Dropout(0.2)(bn2)
out = TimeDistributed(Dense(vocab_size, activation='softmax'))(d)

In [136]:
mdl2 = Model(input=inp, output=out)

In [137]:
mdl2.compile(optimizer=Adam(lr=1e-5), loss='sparse_categorical_crossentropy')

In [138]:
x_stateful = np.stack(np.squeeze(train_data), axis=1)[:12800]
y_stateful = np.atleast_3d(np.stack(output_data, axis=1))[:12800]

x_stateful.shape, y_stateful.shape

((12800, 40), (12800, 40, 1))

In [139]:
n_epoch = 10
for i in range(n_epoch):
    mdl2.reset_states()
    mdl2.fit(x_stateful, y_stateful, nb_epoch=1, batch_size=64, shuffle=False)

Epoch 1/1
12800/12800 [==============================] - 11s - loss: 4.9137    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: 4.4854    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: 4.1476    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: 3.8951    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: 3.6955    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: 3.5340    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: nan    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: nan    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: nan    
Epoch 1/1
12800/12800 [==============================] - 11s - loss: nan    


In [112]:
mdl2.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy')
mdl2.fit(x_stateful, y_stateful, nb_epoch=1, batch_size=64, shuffle=False)

Epoch 1/1
12800/12800 [==============================] - 11s - loss: nan    


In [140]:
def generate_text(num_chars):
    outs = []
    base_str = 'Sufferin'
    for i in range(num_chars):
        prediction = mdl2.predict(np.array([txt_encoder[i] for i in base_str])[np.newaxis])
        next_char = np.argmax(prediction[-1]) # the final model output
        outs.append(txt_decoder[next_char])
        base_str = (base_str + txt_decoder[next_char])[-8:]
    return ''.join(outs)

In [141]:
generate_text(50)

ValueError: Error when checking : expected input_16 to have shape (64, 40) but got array with shape (1, 8)